In [1]:
%time
%load_ext autotime
%load_ext autoreload
%autoreload 2


# if cannot import the modules, add the parent directory to system path might help

import os, tqdm, sys
parent_dir = os.path.abspath(os.getcwd()+'/..')+'/'
sys.path.append(parent_dir)

from utils.save import make_dir, save_pickle, load_pickle, save
from utils.evaluation import spacy_extension

import pandas as pd
import numpy as np
import re
import json
import tqdm

layer1 = json.load(open('/data/yueliu/RecipeAnalytics_201906/raw_data/recipe1M/layer1.json','r'))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 12.9 µs


In [2]:
# data analytics
w = [i for i, recipe in enumerate(layer1) if 'allrecipes' in recipe['url']]
print(' %d recipes are from allrecipes.com' % len(w))

 49010 recipes are from allrecipes.com
time: 456 ms


In [3]:
def clean_line(line):
    '''
    Args:
        line: a string, such as food name, sentences...
    '''
    assert type(line) == str
    
    # all lowercase
    line = line.lower()
    
    # only reserve number and alphabets
    line = re.sub(r'[^a-z0-9+()/?!.,]', ' ', line)
    
    # replace things in brace
    line = re.sub(r'\([^)]*\)', '', line)
    
    # remove extra spaces
    line = re.sub(' +',' ',line).strip()
    return line

def preprocessing(layer1):
    data = []
    for i, recipe in tqdm.tqdm(enumerate(layer1)):
        processed_recipe = {'title':clean_line(recipe['title']), 
                            'id': recipe['id'],
                            'ingredients':[],
                            'instructions':[]}

        field = 'ingredients'
        for line in recipe[field]:
            cleaned = clean_line(line['text'])
            if cleaned:
                processed_recipe[field].append(cleaned)  
        
        field = 'instructions'
        cleaned = clean_line(' '.join([line['text'] for line in recipe[field]]))
        cleaned = ['%s. '% sent for sent in cleaned.split('. ')] 
        processed_recipe[field] = cleaned
        data.append(processed_recipe)
        
    return data

time: 44.2 ms


In [4]:
data = preprocessing(layer1)
save_pickle('../big_data/recipe1M_cleaned.pickle', data, overwrite=True)

1029720it [03:10, 5410.32it/s]


time: 3min 23s


In [5]:
### STEP3 sent to the NYtimes
### assign indices to each ingredient <---> NYtimes
class ny_ingredients:
    def __init__(self, fields):
        # this function will take the global variable ls and dic
        # static & reuseable
        self.ny_ingred = '../../NYtime-parser2/ingred.txt'
        self.ny_result = '../../NYtime-parser2/result.json'
        
        # spacy
        self.fields = fields #['ingredients', 'generated_ingred']
        self.sp = spacy_extension()

    def to_ny(self):
        '''
        using global variables data and ls
        '''
        to_write = []
        for i, v in tqdm.tqdm(enumerate(data)):
            # assing index
            for field in self.fields:
                line_ids = []
                for line in v[field]:
                    if line in to_write:
                        ny_id = to_write.index(line)
                    else:
                        ny_id = len(to_write)
                        to_write.append(line)
                    line_ids.append(ny_id)
                data[i]['ny_%s'%(field)] = line_ids

        # save the file to the folder under NYtime-parser2
        save(filename = self.ny_ingred, 
             to_write = '\n'.join(to_write),
             overwrite = True, 
             print_=True)

        self.to_write = to_write

    def extract(self, ny_ingred):
        '''
        Args: ny_ingred: a list of ingredient names
        '''
        return self.sp.ny_ingred(ny_ingred)

time: 54.2 ms


In [6]:
### start                    
ny_ingr = ny_ingredients(fields = ['ingredients'])
### step 3-1 save it as ingred.txt
ny_ingr.to_ny()
### step 3-2 go to python2 and run NLG_notebooks/Control Nytimes

1029720it [53:30:52,  2.82it/s]


saved ../../NYtime-parser2/ingred.txt
time: 2d 5h 31min 2s


In [8]:
ny_result = pd.read_json(ny_ingr.ny_result)
to_write = []
for i, v in tqdm.tqdm(enumerate(data)):
    for field in ny_ingr.fields:
        temp = [ny_result.loc[ny_id]['name'] for ny_id in v['ny_%s'%(field)]]
        exact, root = ny_ingr.extract(temp)
        data[i]['ny_full_%s'%(field)] = [ny_result.loc[ny_id].to_dict() for ny_id in v['ny_%s'%(field)]]
        data[i]['ny__%s'%(field)] = {'ny':temp, 'exact':exact, 'root':root}

1029720it [7:13:44, 39.57it/s]

time: 7h 13min 59s


In [9]:
save_pickle('../big_data/recipe1M_ny.pickle', data, overwrite=True)

time: 58.1 s
